https://ba.hitomi.la/webp/1686733202/414/18df50f9540ce9c3200c84860f29bdb1bbd4c20cb38becab12624196b5e819e1.webp 怎么构造的？ 
1. 1686733202可能是时间戳，是目前整点的吧,我测试发现17:44时对应时间戳是17:00的
2. 414那部分是将倒数第一位和倒数二三换位后转十六进制int('19e',16)
2. 最后一部分是哈希值，可以通过解析https://ltn.hitomi.la/galleries/2113970.js 获得 其中最后的数字是画廊id
1. aa还是ba有自己的判断逻辑 具体见https://ltn.hitomi.la/gg.js

## 思路
注意以下必须要注意referer 不然会报错！
1. 用户输入画廊链接
2. 提取链接的galleryid
3. 请求js地址，返回数据https://ltn.hitomi.la/galleries/画廊的id.js
3. 请求gg.js,https://ltn.hitomi.la/gg.js 返回映射表，时间戳
4. 解析返回数据，格式化，返回info并存储本地cache
    > 需要注意这步关键是获取哈希值
6. 根据哈希值获取subdomain即十六进制数部分
7. 根据返回映射表判断链接
8. 以上 获取了所有图片的下载链接
9. 下载为jpg格式 可以多线程，只需要传入url

这段是构造是aa还是ba

In [4]:
function subdomain_from_url(url, base) {
    var retval = 'b';
    if (base) {
            retval = base;
    }
    
    var b = 16;
    
    var r = /\/[0-9a-f]{61}([0-9a-f]{2})([0-9a-f])/;
    var m = r.exec(url);
    if (!m) {
            return 'a';
    }
    
    var g = parseInt(m[2]+m[1], b);
    if (!isNaN(g)) {
            retval = String.fromCharCode(97 + gg.m(g)) + retval;
    }
    
    return retval;
}

SyntaxError: invalid syntax (3207787308.py, line 1)